In this notebook I am processing each part of the dataset. For each sample I am extracting the unique triplets and the related trace ids. If the rt is > 0 the relation is considered to be out otherwise it is in . Then I save the result. 

In [1]:
import tarfile
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import zipfile


In [2]:


to_skip = tuple([2,57,60,64,66,75,98,101,125,129,130,144]) # corrupted files
with tqdm(total=145, desc = "Progress") as progress_bar:
    for i in range(0, 145):

        if to_skip.count(i) > 0:
            continue
        # open file
        file = tarfile.open(f'MSCallGraph_{i}.tar.gz')

        # extracting file
        file.extractall('./')

        file.close()
        df = pd.read_csv(f'MSCallGraph_{i}.csv')
        df=df.drop(['Unnamed: 0','timestamp','rpcid','interface',],axis=1)
        df = df.replace('(?)', np.NaN)
        df = df.replace('', np.NaN)
        df = df.replace('NAN', np.NaN)
        df = df.dropna()
        df['io'] = np.NaN
#         display(df)
        df.loc[df['rt']>0, 'io']='out'
        df.loc[df['rt']<0, 'io']='in'
        df = df.dropna()
        df['rpctype'] = df['rpctype'] + '_' + df['io']
        df=df.drop(['io','rt'],axis=1)
        df = df.drop_duplicates()
#         print(df.nunique())
#         display(df)


        df.to_csv(f'MSCallGraph_{i}_io.csv',index=True)

        os.remove(f'MSCallGraph_{i}.csv')
        
        filename = f'MSCallGraph_{i}_io.csv'
        zipfilename = f'MSCallGraph_{i}_io.zip'

        # Create a ZipFile object and add the file to it
        with zipfile.ZipFile(zipfilename, 'w', compresslevel=9, compression=zipfile.ZIP_LZMA) as zip:
            zip.write(filename)
        
        os.remove(f'MSCallGraph_{i}_io.csv')
        progress_bar.update(1)

Progress:  92%|█████████▏| 133/145 [5:21:04<28:58, 144.85s/it]   
